In [1]:
import pandas as pd
import sqlite3

In [2]:
student_info = pd.read_csv('student_info.csv')
subjects = pd.read_csv('subjects.csv')
classes = pd.read_csv('classes.csv')
teacher_subject = pd.read_csv('teacher_subject.csv')
site_user = pd.read_csv('site_user.csv')
teacher_class = pd.read_csv('teacher_class.csv')
user_student = pd.read_csv('user_student.csv')
students = pd.read_csv('students.csv')

In [3]:
conn = sqlite3.connect('database.db')

In [4]:
student_info.drop('Unnamed: 0', axis=1, inplace=True)
subjects.drop('Unnamed: 0', axis=1, inplace=True)
classes.drop('Unnamed: 0', axis=1, inplace=True)
teacher_subject.drop('Unnamed: 0', axis=1, inplace=True)
site_user.drop('Unnamed: 0', axis=1, inplace=True)
teacher_class.drop('Unnamed: 0', axis=1, inplace=True)
user_student.drop('Unnamed: 0', axis=1, inplace=True)
students.drop('Unnamed: 0', axis=1, inplace=True)

In [5]:
student_info.to_sql('student_info', con=conn)  
subjects.to_sql('subjects', con=conn)  
classes.to_sql('classes', con=conn)  
teacher_subject.to_sql('teacher_subject', con=conn)  
site_user.to_sql('site_user', con=conn)  
teacher_class.to_sql('teacher_class', con=conn)  
user_student.to_sql('user_student', con=conn)  
students.to_sql('students', con=conn)  

1270

-- Включаем поддержку внешних ключей
PRAGMA foreign_keys = ON;

-- Удалим временные таблицы, если они остались от прошлых запусков
DROP TABLE IF EXISTS teacher_subject_old;
DROP TABLE IF EXISTS teacher_class_old;
DROP TABLE IF EXISTS user_student_old;
DROP TABLE IF EXISTS students_old;
DROP TABLE IF EXISTS student_info_old;
DROP TABLE IF EXISTS site_user_old;
DROP TABLE IF EXISTS subjects_old;
DROP TABLE IF EXISTS classes_old;

-- =========================
-- 1. Пересоздание subjects
-- =========================
ALTER TABLE subjects RENAME TO subjects_old;

CREATE TABLE subjects (
    subject_id INTEGER PRIMARY KEY,
    name TEXT
);

INSERT INTO subjects (subject_id, name)
SELECT subject_id, name FROM subjects_old;

DROP TABLE subjects_old;

-- =========================
-- 2. Пересоздание site_user
-- =========================
ALTER TABLE site_user RENAME TO site_user_old;

CREATE TABLE site_user (
    user_id INTEGER PRIMARY KEY,
    login TEXT,
    password TEXT,
    type TEXT,
    phone INTEGER,
    email TEXT
);

INSERT INTO site_user (user_id, login, password, type, phone, email)
SELECT user_id, login, password, type, phone, email FROM site_user_old;

DROP TABLE site_user_old;

-- =========================
-- 3. Пересоздание classes
-- =========================
ALTER TABLE classes RENAME TO classes_old;

CREATE TABLE classes (
    class_id INTEGER PRIMARY KEY,
    number INTEGER,
    letter TEXT
);

INSERT INTO classes (class_id, number, letter)
SELECT class_id, number, letter FROM classes_old;

DROP TABLE classes_old;

-- =========================
-- 4. Пересоздание student_info
-- =========================
ALTER TABLE student_info RENAME TO student_info_old;

CREATE TABLE student_info (
    student_id INTEGER PRIMARY KEY,
    surname TEXT,
    name TEXT,
    patronymic TEXT,
    class_id INTEGER,
    FOREIGN KEY (class_id) REFERENCES classes(class_id)
);

-- Удалим строки с некорректными связями
DELETE FROM student_info_old
WHERE class_id NOT IN (SELECT class_id FROM classes);

INSERT INTO student_info (student_id, surname, name, patronymic, class_id)
SELECT student_id, surname, name, patronymic, class_id FROM student_info_old;

DROP TABLE student_info_old;


-- =========================
-- 5. Пересоздание students
-- =========================
ALTER TABLE students RENAME TO students_old;

CREATE TABLE students (
    student_id INTEGER,
    subject_id INTEGER,
    grade INTEGER,
    FOREIGN KEY (subject_id) REFERENCES subjects(subject_id),
    FOREIGN KEY (student_id) REFERENCES student_info(student_id)
);

-- Удалим строки с некорректными связями
DELETE FROM students_old
WHERE subject_id NOT IN (SELECT subject_id FROM subjects)
   OR student_id NOT IN (SELECT student_id FROM student_info);

INSERT INTO students (student_id, subject_id, grade)
SELECT student_id, subject_id, grade FROM students_old;

DROP TABLE students_old;


-- =========================
-- 6. Пересоздание user_student
-- =========================
ALTER TABLE user_student RENAME TO user_student_old;

CREATE TABLE user_student (
    user_id INTEGER,
    student_id INTEGER,
    PRIMARY KEY (user_id, student_id),
    FOREIGN KEY (user_id) REFERENCES site_user(user_id),
    FOREIGN KEY (student_id) REFERENCES student_info(student_id)
);

-- Удалим битые строки
DELETE FROM user_student_old
WHERE user_id NOT IN (SELECT user_id FROM site_user)
   OR student_id NOT IN (SELECT student_id FROM students);

INSERT INTO user_student (user_id, student_id)
SELECT user_id, student_id FROM user_student_old;

DROP TABLE user_student_old;

-- =========================
-- 7. Пересоздание teacher_subject
-- =========================
ALTER TABLE teacher_subject RENAME TO teacher_subject_old;

CREATE TABLE teacher_subject (
    user_id INTEGER,
    subject_id INTEGER,
    PRIMARY KEY (user_id, subject_id),
    FOREIGN KEY (user_id) REFERENCES site_user(user_id),
    FOREIGN KEY (subject_id) REFERENCES subjects(subject_id)
);

DELETE FROM teacher_subject_old
WHERE user_id NOT IN (SELECT user_id FROM site_user)
   OR subject_id NOT IN (SELECT subject_id FROM subjects);

INSERT INTO teacher_subject (user_id, subject_id)
SELECT user_id, subject_id FROM teacher_subject_old;

DROP TABLE teacher_subject_old;

-- =========================
-- 8. Пересоздание teacher_class
-- =========================
ALTER TABLE teacher_class RENAME TO teacher_class_old;

CREATE TABLE teacher_class (
    user_id INTEGER,
    class_id INTEGER,
    PRIMARY KEY (user_id, class_id),
    FOREIGN KEY (user_id) REFERENCES site_user(user_id),
    FOREIGN KEY (class_id) REFERENCES classes(class_id)
);

DELETE FROM teacher_class_old
WHERE user_id NOT IN (SELECT user_id FROM site_user)
   OR class_id NOT IN (SELECT class_id FROM classes);

INSERT INTO teacher_class (user_id, class_id)
SELECT user_id, class_id FROM teacher_class_old;

DROP TABLE teacher_class_old;

-- Включаем обратно внешние ключи
PRAGMA foreign_keys = ON;
